# Capstone project: Fantasy Football Precition

This part of the project contains data inspection, important preparations and
transformations and finally the baseline model is built.


In [1]:
!pip install mlflow
!pip install category_encoders

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.7/24.7 MB 40.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 53.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.0/247.0 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 677.0/677.0 kB 24.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.4/203.4 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.0/120.0 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.6/201.6 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 1.6 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
import mlflow
import mlflow.sklearn
from tqdm.notebook import tqdm

from scipy import sparse
import joblib

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, FunctionTransformer
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

import lightgbm as lgb
import shap

import mlflow.lightgbm

from category_encoders import TargetEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, FunctionTransformer
from sklearn.linear_model import Lasso, LinearRegression
from sklearn.feature_selection import SelectFromModel
from scipy import sparse
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import mlflow

import ast


# Initialize MLflow experiment
mlflow.set_experiment("FPL_Baseline_Regression")

2025/07/31 12:00:33 INFO mlflow.tracking.fluent: Experiment with name 'FPL_Baseline_Regression' does not exist. Creating a new experiment.


<Experiment: artifact_location='file:///content/mlruns/333274811286158441', creation_time=1753963233459, experiment_id='333274811286158441', last_update_time=1753963233459, lifecycle_stage='active', name='FPL_Baseline_Regression', tags={}>

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Data inspection and preparation

In [8]:
# Load the dataset
df = pd.read_csv('/content/drive/MyDrive/Harbour.Space/Diploma Capstone Project/dataset_ver3.2.5.csv')

In [5]:
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 135777 entries, 0 to 135776
Columns: 119 entries, Season to against: npxG/Sh_avg
dtypes: float64(89), int64(21), object(9)
memory usage: 123.3+ MB


,Season,GW,Player,Date,Team,Opposition,Start,Pos,Min,Performance|Gls,...,AstGW,CSGW,Ret,Points,for: Dist_avg,for: npxG_sum,for: npxG/Sh_avg,against: Dist_avg,against: npxG_sum,against: npxG/Sh_avg
0,2017-2018,2,Marko Arnautovic,"[datetime.date(2017, 8, 13)]",WHU,['MUN_A'],['Y'],"['LW,LM']",90.0,0,...,0.0,0.0,0.0,-2.0,17.8,0.5,0.05,18.3,2.1,0.10
1,2017-2018,2,Ryan Bertrand,"[datetime.date(2017, 8, 12)]",SOU,['SWA_H'],['Y'],['LB'],85.0,0,...,0.0,0.0,0.0,1.0,17.8,2.0,0.07,15.6,0.3,0.07
2,2017-2018,2,Abdoulaye Doucoure,"[datetime.date(2017, 8, 12)]",WAT,['LIV_H'],['Y'],['CM'],90.0,1,...,0.0,1.0,0.0,3.0,15.9,2.1,0.25,13.9,1.8,0.14
3,2017-2018,2,Jonathan Walters,"[datetime.date(2017, 8, 12)]",BUR,['CHE_A'],['N'],['FW'],16.0,0,...,0.0,0.0,0.0,1.0,14.5,0.6,0.07,21.3,1.5,0.08
4,2017-2018,2,Antonio Valencia,"[datetime.date(2017, 8, 13)]",MUN,['WHU_H'],['Y*'],['RB'],90.0,0,...,0.0,1.0,1.0,6.0,18.3,2.1,0.10,17.8,0.5,0.05


In [6]:
df.columns

Index(['Season', 'GW', 'Player', 'Date', 'Team', 'Opposition', 'Start', 'Pos',
       'Min', 'Performance|Gls',
       ...
       'AstGW', 'CSGW', 'Ret', 'Points', 'for: Dist_avg', 'for: npxG_sum',
       'for: npxG/Sh_avg', 'against: Dist_avg', 'against: npxG_sum',
       'against: npxG/Sh_avg'],
      dtype='object', length=119)

#### Mapping for positions

In [7]:
'''# Helpers (with ‘MF’ included)
def parse_pos_list(x):
    s = str(x).strip()
    if s.startswith('[') and s.endswith(']'):
        try:
            raw_list = ast.literal_eval(s)
        except:
            raw_list = []
        codes = []
        for item in raw_list:
            if isinstance(item, str) and ',' in item:
                codes.extend([p.strip() for p in item.split(',') if p.strip()])
            elif isinstance(item, str):
                codes.append(item.strip())
        return codes or raw_list
    if ',' in s:
        return [p.strip() for p in s.split(',') if p.strip()]
    return [s]

GK_SET  = {'GK'}
DEF_SET = {'LB','LWB','CB','RB','RWB','WB'}
MID_SET = {'CDM','CM','CAM','LM','RM','DM','AM','MF'}  # added 'MF'
FWD_SET = {'ST','CF','LF','RF','LW','RW','FW'}

def map_macro(pos_list):
    if any(p in GK_SET  for p in pos_list): return 'GK'
    if any(p in DEF_SET for p in pos_list): return 'DEF'
    if any(p in MID_SET for p in pos_list): return 'MID'
    if any(p in FWD_SET for p in pos_list): return 'FWD'
    return 'OTHER'

# Samples to test
samples = [
    "['MF','MF']",
    "['DM','CM']",
    "['LW,LM']",
    "['CB,WB']",
    "['RW,RM']"
]

for s in samples:
    codes = parse_pos_list(s)
    macro = map_macro(codes)
    print(f"{s} → parsed: {codes} → macro: {macro}")
'''

['MF','MF'] → parsed: ['MF', 'MF'] → macro: MID
['DM','CM'] → parsed: ['DM', 'CM'] → macro: MID
['LW,LM'] → parsed: ['LW', 'LM'] → macro: MID
['CB,WB'] → parsed: ['CB', 'WB'] → macro: DEF
['RW,RM'] → parsed: ['RW', 'RM'] → macro: MID


In [9]:
# Apply final Pos mapping to df

# Helpers for parsing and mapping
def parse_pos_list(x):
    s = str(x).strip()
    # Python-list literal?
    if s.startswith('[') and s.endswith(']'):
        try:
            raw_list = ast.literal_eval(s)
        except:
            raw_list = []
        codes = []
        for item in raw_list:
            if isinstance(item, str) and ',' in item:
                codes.extend([p.strip() for p in item.split(',') if p.strip()])
            elif isinstance(item, str):
                codes.append(item.strip())
        return codes or raw_list
    # Comma-separated string
    if ',' in s:
        return [p.strip() for p in s.split(',') if p.strip()]
    # Single code
    return [s]

# Define macro-role sets
GK_SET  = {'GK'}
DEF_SET = {'LB', 'LWB', 'CB', 'RB', 'RWB', 'WB'}
MID_SET = {'CDM', 'CM', 'CAM', 'LM', 'RM', 'DM', 'AM', 'MF'}  # included 'MF'
FWD_SET = {'ST', 'CF', 'LF', 'RF', 'LW', 'RW', 'FW'}

def map_macro(pos_list):
    if any(p in GK_SET  for p in pos_list):
        return 'GK'
    if any(p in DEF_SET for p in pos_list):
        return 'DEF'
    if any(p in MID_SET for p in pos_list):
        return 'MID'
    if any(p in FWD_SET for p in pos_list):
        return 'FWD'
    return 'OTHER'


In [10]:
# Apply to the DataFrame
df['Pos_list'] = df['Pos'].apply(parse_pos_list)
df['Pos']      = df['Pos_list'].apply(map_macro)
df.drop(columns=['Pos_list'], inplace=True)

# Verify distribution
print("Post-mapping Pos distribution:")
print(df['Pos'].value_counts())



Post-mapping Pos distribution:
Pos
DEF    53686
MID    52062
FWD    20845
GK      9184
Name: count, dtype: int64


## Baseline construction

### Idea

We'll take a few main features for it:

*  **Min** – Minutes played in the previous match (proxy for selection likelihood and match fitness).

*  **GlsPrev, AstPrev, CSPrev** – Simple rolling “last-GW” counts of goals, assists, and clean sheets. They’re straightforward form indicators that don’t require aggregating many weeks.

*  **PrevPoints** – The total fantasy points from last GW. A compact form summary that blends all scoring actions into one number.

*  **Price** – Player cost; high-priced players tend to be starters on strong teams, low risk of rotation, and often higher scorers.

*  **Ownership, %** – Crowd wisdom: if 50%+ of managers own a player, that often correlates with consistent point returns.

*  **Selected** – Raw number of managers who selected the player (absolute rather than percentage)—another popularity/form proxy.

Simple idea - for now let's avoid complex feature engineering.
Low dimensional approach, so a quick sanity-check model can run in seconds.

#### feature construction

In [11]:
NUMERIC_FEATS = ['Min','GlsPrev','AstPrev','CSPrev','PrevPoints','Price','Ownership, %','Selected']
CAT_FEATS     = ['Team','Opposition']
POS_FEAT      = ['Pos']

In [12]:
# 0) Coerce Points (and Ret) to numeric, so any non‐numeric strings become NaN
df['Points'] = pd.to_numeric(df['Points'], errors='coerce')
df['Ret']    = pd.to_numeric(df['Ret'],    errors='coerce')

# 1) Count missing targets
missing_pts = df['Points'].isna().sum()
print(f"Missing Points before drop: {missing_pts}")

# 2) If any remain, preview a few to see what’s going on
if missing_pts > 0:
    display(df[df['Points'].isna()].head())

# 3) Drop rows where Points is NaN
initial_len = len(df)
df = df.dropna(subset=['Points'])
print(f"Dropped {initial_len - len(df)} rows with NaN Points")

# 4) (Optional) Impute any remaining numeric NaNs with column means
numeric_cols = NUMERIC_FEATS + [c for c in df.columns if c.startswith('against: ')]
for col in numeric_cols:
    n = df[col].isna().sum()
    if n:
        mean_val = df[col].mean()
        df[col].fillna(mean_val, inplace=True)
        print(f"Imputed {n} NaNs in {col} → mean {mean_val:.3f}")

# 5) Final check
assert df['Points'].isna().sum() == 0, "Still have missing Points!"
print("Cleaned: no NaNs remain in Points.")



Missing Points before drop: 4998


,Season,GW,Player,Date,Team,Opposition,Start,Pos,Min,Performance|Gls,...,AstGW,CSGW,Ret,Points,for: Dist_avg,for: npxG_sum,for: npxG/Sh_avg,against: Dist_avg,against: npxG_sum,against: npxG/Sh_avg
7591,2017-2018,21,Marko Arnautovic,"[datetime.date(2017, 12, 16), datetime.date(20...",WHU,"['STK_A', 'NEW_H', 'BOU_A']","['Y', 'Y', 'Y']",MID,258.0,4,...,NaN,NaN,NaN,NaN,16.6,5.3,0.15,18.0,5.7,0.11
7618,2017-2018,21,Javier Hernandez,"[datetime.date(2017, 12, 16), datetime.date(20...",WHU,"['STK_A', 'NEW_H', 'BOU_A']","['N', 'N', 'N']",MID,54.0,0,...,NaN,NaN,NaN,NaN,16.6,5.3,0.15,18.0,5.7,0.11
7631,2017-2018,21,Declan Rice,"[datetime.date(2017, 12, 3), datetime.date(201...",WHU,"['MCI_A', 'STK_A', 'BOU_A']","['Y', 'N', 'N']",DEF,158.0,0,...,NaN,NaN,NaN,NaN,16.6,5.3,0.15,18.0,5.7,0.11
7641,2017-2018,21,Kieran Trippier,"[datetime.date(2017, 12, 2), datetime.date(201...",TOT,"['WAT_A', 'STK_H', 'MCI_A']","['Y', 'Y', 'Y']",DEF,270.0,0,...,NaN,NaN,NaN,NaN,18.8,5.5,0.12,17.1,3.6,0.08
7643,2017-2018,21,Harry Kane,"[datetime.date(2017, 12, 16), datetime.date(20...",TOT,"['MCI_A', 'BUR_A', 'SOU_H']","['Y', 'Y', 'Y']",FWD,270.0,6,...,NaN,NaN,NaN,NaN,18.8,5.5,0.12,17.1,3.6,0.08


Dropped 4998 rows with NaN Points
Cleaned: no NaNs remain in Points.


In [13]:
# I also want to capture opposition perfomance

opp_perf_feats = [c for c in df.columns if c.startswith('against: ')]
print("Found opposition-performance columns:", opp_perf_feats)

NUMERIC_FEATS += opp_perf_feats
print("Updated NUMERIC_FEATS:", NUMERIC_FEATS)

Found opposition-performance columns: ['against: Dist_avg', 'against: npxG_sum', 'against: npxG/Sh_avg']
Updated NUMERIC_FEATS: ['Min', 'GlsPrev', 'AstPrev', 'CSPrev', 'PrevPoints', 'Price', 'Ownership, %', 'Selected', 'against: Dist_avg', 'against: npxG_sum', 'against: npxG/Sh_avg']


In [17]:
# Splits - each season would be splitted:
# Train: GameWeeks 1-30, TEST: GameWeeks 31-38

train_rr = df[df.GW <= 30]
test_rr  = df[(df.GW >= 31) & (df.GW <= 38)]

# probably will be used later for time-series approach
train_ts = df[df.GW <= 26]
val_ts   = df[(df.GW >= 27) & (df.GW <= 30)]

In [19]:
ohe_check = ColumnTransformer([
    ('ohe_cat',
     OneHotEncoder(drop='first', sparse_output=True, handle_unknown='ignore'),
     CAT_FEATS + POS_FEAT)
], remainder='passthrough')

X_tr = train_rr[NUMERIC_FEATS + CAT_FEATS + POS_FEAT]

X_enc = ohe_check.fit_transform(X_tr)

n_samples, n_features = X_enc.shape
print(f"Encoded training data shape: {n_samples} samples × {n_features} features")

Encoded training data shape: 99035 samples × 15383 features


In [ ]:
# Build the tiny pipeline up front
preproc = ColumnTransformer([
    # target-encode Team & Opposition → 2 dims
    ('te', TargetEncoder(cols=CAT_FEATS), CAT_FEATS),
    # one-hot only Pos → 3 dims (GK,DEF,MID,FWD drop_first)
    ('pos', OneHotEncoder(drop='first', sparse_output=True, handle_unknown='ignore'), POS_FEAT),
    # passthrough all numeric feats (including your against: ones)
    ('num', 'passthrough', NUMERIC_FEATS)
], remainder='drop')

In [21]:
# reduce dimensionality

pipeline_small2 = Pipeline([
    ('preproc', preproc),
    ('to_sparse', FunctionTransformer(lambda X: sparse.csr_matrix(X), accept_sparse=True))
])

# 3) Fit & transform the train split
X_tr = train_rr[NUMERIC_FEATS + CAT_FEATS + POS_FEAT]
y_tr = train_rr['Points']
X_enc = pipeline_small2.fit_transform(X_tr, y_tr)

# 4) Print the new shape
print(f"Reduced-encoded training shape: {X_enc.shape[0]} samples × {X_enc.shape[1]} features")



Reduced-encoded training shape: 99035 samples × 16 features


In [22]:
# — Cell: Train & evaluate Lasso on 16-feature pipeline (fixed RMSE calculation) —

pipe_small2_full = Pipeline([
    ('preproc', preproc),
    ('to_sparse', FunctionTransformer(lambda X: sparse.csr_matrix(X), accept_sparse=True)),
    ('select', SelectFromModel(
        Lasso(alpha=0.01, tol=1e-3, max_iter=500, random_state=42),
        max_features=15,
        threshold=-np.inf
    )),
    ('model', LinearRegression())
])


In [23]:
with mlflow.start_run(run_name="Lasso_Small2_Final"):
    mlflow.log_params({
        "model_type": "Lasso_Small2",
        "alpha": 0.01,
        "tol": 1e-3,
        "max_iter": 500,
        "max_features": 15
    })

    # fit
    X_tr = train_rr[NUMERIC_FEATS + CAT_FEATS + POS_FEAT]
    y_tr = train_rr['Points']
    pipe_small2_full.fit(X_tr, y_tr)

    # predict & evaluate
    X_te = test_rr[NUMERIC_FEATS + CAT_FEATS + POS_FEAT]
    y_te = test_rr['Points']
    preds = pipe_small2_full.predict(X_te)

    mae = mean_absolute_error(y_te, preds)
    mse = mean_squared_error(y_te, preds)
    rmse = np.sqrt(mse)
    r2  = r2_score(y_te, preds)

    mlflow.log_metrics({"MAE": mae, "RMSE": rmse, "R2": r2})
    print(f"Lasso_Small2 → MAE: {mae:.4f}, RMSE: {rmse:.4f}, R2: {r2:.4f}")


Lasso_Small2 → MAE: 1.8110, RMSE: 2.7172, R2: 0.1064


In [44]:
# 1) Define a top-level function for sparse conversion


def to_csr(X):
    return sparse.csr_matrix(X)

# 2) Rebuild pipeline using that function (no lambdas)
pipe_small2_picklable = Pipeline([
    ('preproc', preproc),
    ('to_sparse', FunctionTransformer(func=to_csr, accept_sparse=True)),
    ('select', SelectFromModel(
        Lasso(alpha=0.01, tol=1e-3, max_iter=500, random_state=42),
        max_features=15, threshold=-np.inf
    )),
    ('model', LinearRegression())
])

# 3) Transfer fitted preproc & selector params into the new pipeline
#    We only need to copy fitted attributes, not refit everything:
pipe_small2_picklable.named_steps['preproc'] = pipe_small2_full.named_steps['preproc']
pipe_small2_picklable.named_steps['select']  = pipe_small2_full.named_steps['select']
pipe_small2_picklable.named_steps['model']   = pipe_small2_full.named_steps['model']

# 4) pickle it
joblib.dump(pipe_small2_picklable, 'lasso_small2_full_model.pkl')
print("Model saved to lasso_small2_full_model.pkl")


Model saved to lasso_small2_full_model.pkl


In [ ]:
# TODO: call MLFLOW

In [26]:
# Top-10 players by total Points in the test set (GWs 31–38) for each season
top10_per_season = (
    test_rr
    .groupby(['Season','Player'])['Points']
    .sum()
    .reset_index()
    .sort_values(['Season','Points'], ascending=[True, False])
)

for season, grp in top10_per_season.groupby('Season'):
    print(f"Season {season} — Top 10 players:")
    display(grp.head(10))


Season 2017-2018 — Top 10 players:


,Season,Player,Points
354,2017-2018,Mohamed Salah,66.0
401,2017-2018,Pierre-Emerick Aubameyang,61.0
496,2017-2018,Wilfried Zaha,60.0
52,2017-2018,Ayoze Perez,55.0
385,2017-2018,Patrick van Aanholt,55.0
322,2017-2018,Marko Arnautovic,51.0
403,2017-2018,Raheem Sterling,50.0
219,2017-2018,Jamie Vardy,48.0
306,2017-2018,Luka Milivojevic,48.0
91,2017-2018,Christian Eriksen,46.0


Season 2018-2019 — Top 10 players:


,Season,Player,Points
566,2018-2019,Aymeric Laporte,64.0
981,2018-2019,Trent Alexander-Arnold,62.0
936,2018-2019,Sadio Mane,58.0
656,2018-2019,Eden Hazard,56.0
867,2018-2019,Mohamed Salah,56.0
812,2018-2019,Lucas Digne,55.0
945,2018-2019,Seamus Coleman,53.0
729,2018-2019,Jamie Vardy,52.0
930,2018-2019,Ryan Babel,49.0
567,2018-2019,Ayoze Perez,47.0


Season 2019-2020 — Top 10 players:


...

,Season,Player,Points
3220,2023-2024,Cole Palmer,87.0
3355,2023-2024,Jean-Philippe Mateta,75.0
3390,2023-2024,Josko Gvardiol,70.0
3395,2023-2024,Kai Havertz,67.0
3533,2023-2024,Phil Foden,67.0
3126,2023-2024,Alexander Isak,62.0
3273,2023-2024,Erling Haaland,61.0
3189,2023-2024,Bruno Fernandes,59.0
3190,2023-2024,Bruno Guimaraes,58.0
3173,2023-2024,Ben White,57.0


Season 2024-2025 — Top 10 players:


,Season,Player,Points
3884,2024-2025,Jarrod Bowen,66.0
3787,2024-2025,Eberechi Eze,60.0
3859,2024-2025,Jacob Murphy,53.0
3679,2024-2025,Antoine Semenyo,52.0
3839,2024-2025,Harvey Barnes,52.0
3809,2024-2025,Fabian Schar,51.0
3957,2024-2025,Leandro Trossard,51.0
3717,2024-2025,Bryan Mbeumo,49.0
4195,2024-2025,Yoane Wissa,48.0
4054,2024-2025,Nick Pope,44.0


from matplotlib import pyplot as plt
_df_0['Points'].plot(kind='hist', bins=20, title='Points')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_1['Points'].plot(kind='line', figsize=(8, 4), title='Points')
plt.gca().spines[['top', 'right']].set_visible(False)

In [34]:
# Define test set (GWs 31–38)
test_rr = df[(df.GW >= 31) & (df.GW <= 38)]

# Total points per player in the test period
player_totals = test_rr.groupby('Player')['Points'].sum()

# Keep only players with > <points_threshold> total points
points_threshold = 76
active_players = player_totals[player_totals > points_threshold].index
filtered_rr = test_rr[test_rr['Player'].isin(active_players)]

print(f"Removed {len(player_totals) - len(active_players)} players with ≤{points_threshold} total pts; {len(active_players)} remain.")


Removed 1336 players with ≤76 total pts; 236 remain.


In [35]:
# OLD ---

# Compute per‐GW median and mean Points by position for the filtered set
stats_by_pos = (
    filtered_rr
    .groupby('Pos')['Points']
    .agg(median='median', mean='mean')
)

print(f"Median and mean Points per GW by position (players > {points_threshold} total pts):")
print(stats_by_pos)


Median and mean Points per GW by position (players > 76 total pts):
     median      mean
Pos                  
DEF     1.0  2.445140
FWD     2.0  3.619565
GK      2.0  3.381760
MID     2.0  2.924480


In [ ]:
# ----
# Cell: Compute mode & median Points per GW by position (players >76 total pts) and save to CSV

# 1) Define test split and filter active players
test_rr = df[(df.GW >= 31) & (df.GW <= 38)]
player_totals = test_rr.groupby('Player')['Points'].sum()
active_players = player_totals[player_totals > 76].index
filtered_rr = test_rr[test_rr['Player'].isin(active_players)]


In [39]:
# Cell: Compute mean & median (floats) by position and save to CSV

stats_by_pos = (
    filtered_rr
    .groupby('Pos')['Points']
    .agg(
        median='median',
        mean='mean'
    )
)

In [40]:
print(f"Median and mode Points per GW by position (players > {points_threshold} total pts):")
print(stats_by_pos)

Median and mode Points per GW by position (players > 76 total pts):
     median      mean
Pos                  
DEF     1.0  2.445140
FWD     2.0  3.619565
GK      2.0  3.381760
MID     2.0  2.924480


In [41]:
stats_by_pos.to_csv('median_mean_points_by_pos_filtered.csv')
print("Saved median & mean (float) points by position to 'median_mean_points_by_pos_filtered.csv'")

Saved median & mean (float) points by position to 'median_mean_points_by_pos_filtered.csv'
